# Emerging Technologies

This notebook contains solutions for the assessment problems on classical and quantum algorithms using the Deutsch–Jozsa algorithm.


## Import Required

In [29]:
import random

## Problem 1: Generating Random Boolean Functions

**Instructions:**  
Write a Python function `random_constant_balanced` that returns a randomly chosen function from the set of constant or balanced functions taking four Boolean inputs.

In [30]:
# Problem 1 solution goes here
# Deterministic operations
def random_constant_balanced():
    """
    Returns a randomly chosen constant or balanced function
    that takes 4 Boolean inputs.
    """
    # Step 1: Randomly decide constant or balanced
    is_constant = random.choice([True, False])
    
    if is_constant:
        # Step 2a: Constant function - all 16 outputs are the same
        value = random.choice([0, 1])
        outputs = [value] * 16
    else:
        # Step 2b: Balanced function - 8 zeros and 8 ones, shuffled
        outputs = [0] * 8 + [1] * 8
        random.shuffle(outputs)
    
    # Step 3: Return a function that uses this lookup table
    def f(a, b, c, d):
        # Convert 4 bits to index (0-15)
        index = 8*a + 4*b + 2*c + d
        return outputs[index]
    
    return f

In [31]:
# Test the function
f = random_constant_balanced()

# Print all outputs to see the pattern
print("Input -> Output")
for i in range(16):
    a = (i >> 3) & 1
    b = (i >> 2) & 1
    c = (i >> 1) & 1
    d = i & 1
    print(f"({a},{b},{c},{d}) -> {f(a,b,c,d)}")

# Count outputs to verify constant/balanced
outputs = [f((i>>3)&1, (i>>2)&1, (i>>1)&1, i&1) for i in range(16)]
zeros = outputs.count(0)
ones = outputs.count(1)
print(f"\nZeros: {zeros}, Ones: {ones}")
print(f"Type: {'Constant' if zeros == 0 or zeros == 16 else 'Balanced'}")

Input -> Output
(0,0,0,0) -> 1
(0,0,0,1) -> 1
(0,0,1,0) -> 0
(0,0,1,1) -> 1
(0,1,0,0) -> 0
(0,1,0,1) -> 1
(0,1,1,0) -> 0
(0,1,1,1) -> 0
(1,0,0,0) -> 0
(1,0,0,1) -> 1
(1,0,1,0) -> 0
(1,0,1,1) -> 0
(1,1,0,0) -> 1
(1,1,0,1) -> 1
(1,1,1,0) -> 1
(1,1,1,1) -> 0

Zeros: 8, Ones: 8
Type: Balanced


## Problem 2: Classical Testing for Function Type

**Instructions:**  
Write a Python function `determine_constant_balanced(f)` that analyzes a function f and returns "constant" or "balanced". Include a brief note on efficiency.

In [32]:
# Problem 2 solution goes here

## Problem 3: Quantum Oracles

**Instructions:**  
Using Qiskit, create quantum oracles for each of the possible single-Boolean-input functions in Deutsch's algorithm. Explain how each oracle works.

In [33]:
# Problem 3 solution goes here

## Problem 4: Deutsch's Algorithm with Qiskit

**Instructions:**  
Design a quantum circuit that solves Deutsch's problem for single-input functions. Demonstrate its use with each oracle and explain the interference pattern.

In [34]:
# Problem 4 solution goes here

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm

**Instructions:**  
Use Qiskit to create a quantum circuit for four-bit functions generated in Problem 1. Demonstrate it on constant and balanced functions and explain the results.

In [35]:
# Problem 5 solution goes here